In [38]:
# Import libraries and classes required 
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd 

In [9]:
#! pip install scikit-learn

In [105]:
#import data
# Import dataset:
dataset = pd.read_excel("union_vif_ds.xlsx")
dataset.shape

(34326, 16)

In [106]:
dataset.columns

Index(['Unnamed: 0', 'numero_documento', 'fecha_denuncia', 'ocupacion', 'edad',
       'testigo', 'arma_medio', 'modalidad_1257', 'modalidad_294',
       'nivel_academico', 'sexo', 'Número denuncias vif', 'relato', 'DS',
       'Homicido', 'reincidente total'],
      dtype='object')

In [66]:
#drop na values
dataset2 = dataset[['arma_medio',  'modalidad_294', 'sexo', 'nivel_academico', 'edad', 'reincidente total']].dropna().reset_index(drop=True)
dataset2.shape

(29828, 6)

In [88]:
X = dataset2[['arma_medio',  'modalidad_294', 'sexo', 'nivel_academico', 'edad']]
#X = dataset2[['arma_medio',  'modalidad_294', 'sexo', 'nivel_academico']]
y = dataset2['reincidente total']

In [77]:
X['nivel_academico'].unique()

array(['Bachillerato', 'Universitario', 'Primaria', 'Técnica',
       'Especialización'], dtype=object)

In [96]:
# Define encoders for categorical variables
ohe = OneHotEncoder(sparse=False)
oe = OrdinalEncoder(categories=[[ "Primaria", "Bachillerato", "Técnica", "Universitario", "Especialización"]])

# Define column transformer to apply encoders to categorical variables
ct = make_column_transformer(
    (ohe, ['arma_medio', 'modalidad_294', 'sexo']),
    (oe, ['nivel_academico']),
    remainder='passthrough')

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Fit the column transformer on the training data
ct.fit(X_train)

# Apply the column transformer to the training and testing data
X_train_enc = ct.transform(X_train)
X_test_enc = ct.transform(X_test)

# Create logistic regression object
logreg = LogisticRegression()

# Fit the logistic regression model on the encoded training data
logreg.fit(X_train_enc, y_train)

# Predict on the encoded test data
y_pred = logreg.predict(X_test_enc)

# Evaluate the model performance
accuracy = logreg.score(X_test_enc, y_test)

# View the coefficients of the logistic regression model
coefficients = logreg.coef_
print(coefficients)

# Get the confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
print(confusion_mat)


/usr/local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


[[ 0.0841939  -0.18623943  0.1837153  -0.37116517  0.17724036  0.04341387
   0.03899179 -0.15050513 -0.10833953 -0.09803847  0.11990399 -0.16692131
  -0.16821292  0.36369069  0.10058971 -0.26130037  0.1924592  -0.10313944]]


/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
